In [1]:
import pandas as pd
import psycopg2

In [14]:
conn = psycopg2.connect("dbname='cap' user='postgres' host='ec2-52-27-114-159.us-west-2.compute.amazonaws.com' port=9000 password ='secret'")
data = pd.read_sql_query("SELECT * FROM nlp_dim", conn)

In [15]:
data.head()

,article_id,tokenized_body,word_count,stopworded_body,lemmatized_body,word_bag,named_entities,lexical_diversity
0,1,"{Story,highlights,Attack,at,the,heart,of,UK,'s...",369,"{story,highlights,attack,heart,uk,capital,late...","{story,highlight,attack,heart,uk,capital,lates...","{""(attack,8)"",""(london,5)"",""(state,4)"",""(terro...","{""(UK,LOCATION)"",""(\""Peter Bergen\"",PERSON)"",""...",73.423423
1,2,"{(,CNN,),The,perfect,bar,"","",like,the,perfect,...",3145,"{cnn,the,perfect,bar,like,perfect,cocktail,car...","{cnn,the,perfect,bar,like,perfect,cocktail,car...","{""(:,71)"",""(belfast,57)"",""(the,47)"",""(``,46)"",...","{""(\""New York\"",LOCATION)"",""(\""Dead Rabbit Gro...",39.760292
2,4,"{A,fundraiser,for,the,family,of,the,police,off...",545,"{a,fundraiser,family,police,officer,killed,wed...","{a,fundraiser,family,police,officer,kill,wedne...","{""(“,11)"",""(palmer,8)"",""(”,8)"",""(police,6)"",""(...",{},63.311688
3,5,"{Seven,Words,the,Feds,Ca,n't,Say,Shortly,after...",756,"{seven,words,feds,ca,n't,say,shortly,donald,tr...","{seven,word,fed,ca,n't,say,shortly,donald,trum...","{""(climate,30)"",""(change,26)"",""(little,20)"",""(...","{""(\""Donald Trump\"",PERSON)"",""(\""Trump Blocks ...",42.710472
4,6,"{When,a,patient,arrives,at,Bayonne,Hospital,Ce...",2507,"{when,patient,arrives,bayonne,hospital,center,...","{when,patient,arrive,bayonne,hospital,center,n...","{""(hospital,47)"",""(health,35)"",""(price,34)"",""(...","{""(\""Bayonne Hospital Center\"",LOCATION)"",""(\""...",50.708024


## TF IDF outline 

- list of articles bodies 
- feed list into function to make a sparse matrix for training
- use that with tf-idf stuff above to create a tf-idf transformer 
- use that to label every individual article with tf-idf vectors in a database table


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
article_id           1000 non-null int64
tokenized_body       1000 non-null object
word_count           1000 non-null int64
stopworded_body      1000 non-null object
lemmatized_body      1000 non-null object
word_bag             1000 non-null object
named_entities       1000 non-null object
lexical_diversity    1000 non-null float64
dtypes: float64(1), int64(2), object(5)
memory usage: 62.6+ KB


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# transforms data into tfidf matrix representation
vectorizer = TfidfVectorizer(max_df=0.5, max_features=100,
                                 min_df=2, use_idf=True)


In [18]:
# fit our data (list of article bodies) to a tfidf representation
X = vectorizer.fit_transform(data.lemmatized_body)

# verify we have a sparse matrix of 100 tfidf features for each article 
# should be 5*100 sparse matrix
X

<1000x100 sparse matrix of type '<class 'numpy.float64'>'
	with 29546 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.cluster import KMeans

# How many clusters we want
true_k = 4

# create the KMeans object with initial settings
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=True)

In [20]:
# fit our tfidf data to the kmeans model
km.fit(X)

Initialization complete
Iteration  0, inertia 1344.736
Iteration  1, inertia 740.625
Iteration  2, inertia 726.899
Iteration  3, inertia 719.020
Iteration  4, inertia 715.689
Iteration  5, inertia 714.076
Iteration  6, inertia 713.257
Iteration  7, inertia 712.337
Iteration  8, inertia 711.677
Iteration  9, inertia 711.550
Iteration 10, inertia 711.405
Iteration 11, inertia 711.062
Iteration 12, inertia 710.680
Iteration 13, inertia 710.218
Iteration 14, inertia 709.881
Iteration 15, inertia 709.752
Iteration 16, inertia 709.715
Iteration 17, inertia 709.694
Iteration 18, inertia 709.649
Iteration 19, inertia 709.575
Iteration 20, inertia 709.553
Iteration 21, inertia 709.547
Iteration 22, inertia 709.521
Iteration 23, inertia 709.501
Iteration 24, inertia 709.481
Converged at iteration 24: center shift 0.000000e+00 within tolerance 8.121001e-07


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=4, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [21]:
terms = vectorizer.get_feature_names()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
labels = km.labels_

for i in range(3):
    for ind in order_centroids[i, :3]:
            print(' %s' % terms[ind], end='')

 image get like bill obamacare house trump president state

In [22]:
for l in labels:
    print(l)

3
0
3
0
1
1
0
0
2
3
2
3
0
2
2
2
0
0
0
1
3
3
0
0
0
0
0
0
0
0
2
0
0
3
0
0
0
0
0
0
0
3
0
2
3
0
1
3
3
0
0
0
0
1
3
3
1
2
1
3
1
3
3
3
2
1
1
3
3
0
3
3
1
2
3
1
1
2
3
2
0
0
3
1
3
0
3
1
2
3
3
2
2
2
2
3
2
0
3
2
3
3
1
0
3
1
3
3
2
2
3
0
2
1
3
2
2
1
3
2
3
2
3
1
3
3
2
1
2
3
0
1
2
3
3
3
2
3
1
3
1
1
2
3
3
0
1
1
3
1
3
1
0
3
3
1
3
3
3
3
3
2
2
3
0
3
1
3
0
1
3
1
0
3
2
2
1
2
3
1
0
0
3
2
3
3
1
3
0
1
3
2
2
3
3
0
3
2
1
1
3
3
3
3
2
1
0
3
3
3
2
3
3
2
3
3
0
2
3
3
3
3
2
0
3
0
2
0
2
0
1
3
0
1
0
2
3
2
0
2
1
0
0
0
0
1
2
0
0
1
0
0
1
1
2
0
1
3
0
0
2
3
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
3
2
2
1
0
0
0
0
3
0
0
0
3
2
0
0
0
3
0
3
2
0
0
0
0
0
3
0
0
0
0
0
0
3
0
3
0
0
2
0
0
0
0
0
0
0
2
0
3
0
0
3
1
2
0
0
3
2
3
0
1
3
0
0
0
1
2
3
0
0
0
1
1
0
3
0
0
0
0
3
2
3
3
3
0
3
3
1
1
1
3
1
3
2
3
3
0
1
3
0
1
2
3
3
3
0
3
0
0
3
3
2
1
0
2
3
3
2
1
1
3
3
0
3
2
3
1
2
3
1
3
1
1
3
3
1
3
0
3
3
0
3
0
3
3
2
0
1
3
0
3
1
3
0
3
3
2
3
2
3
0
2
0
3
0
2
3
1
3
1
3
1
3
3
3
3
3
3
3
1
3
1
2
3
1
1
3
2
3
0
3
3
0
1
2
1
3
3
3
2
1
3
3
1
0
2
1
1
3
2
1
0
3
0
0
0
0
0


In [24]:
tfidf = TfidfVectorizer(max_features=100)
X_test = tfidf.fit_transform([data.lemmatized_body[0]])

In [25]:
km.predict(X_test)

array([3], dtype=int32)